In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def g(data):
    return pd.read_csv(StringIO(data))

In [23]:
d="""date,data
2019-01-03,10
2019-01-04,20
2019-05-21,30
2019-05-22,40
2020-10-15,50
2020-10-16,60
2021-04-04,70
2021-04-05,80"""
df=g(d)
df
df['date'] = pd.to_datetime(df.date)
df

,date,data
0,2019-01-03,10
1,2019-01-04,20
2,2019-05-21,30
3,2019-05-22,40
4,2020-10-15,50
5,2020-10-16,60
6,2021-04-04,70
7,2021-04-05,80


In [31]:
#df['year'] = pd.to_datetime(df.date).dt.isocalendar()
df[['year','week']] = pd.to_datetime(df.date).dt.isocalendar()[['year', 'week']]
pd.pivot_table(data = df,index= 'year', columns = 'week',values = 'data', aggfunc = np.mean )

week,1,13,14,21,42
year,,,,,
2019,15.0,NaN,NaN,35.0,NaN
2020,NaN,NaN,NaN,NaN,55.0
2021,NaN,70.0,80.0,NaN,NaN


In [38]:
df=pd.DataFrame({
    'mydate1':['17JUL2019:00:00:00','18MAR2019:00:00:00','27MAR2019:00:00:00'],
    'mydate2':['31DEC2019','31Dec2019','31Dec2019'],
    'curmth':[0,0,1]
})
df.mydate1 = pd.to_datetime(df.mydate1, format='%d%b%Y:%H:%M:%S')
df.mydate2 = pd.to_datetime(df.mydate2, format='%d%b%Y', )
df

,mydate1,mydate2,curmth
0,2019-07-17,2019-12-31,0
1,2019-03-18,2019-12-31,0
2,2019-03-27,2019-12-31,1


In [39]:
#mnth_diff = (df.mydate2 - df.mydate1)//np.timedelta64(1,'M')
mnth_diff = df.mydate2.dt.month - df.mydate1.dt.month
df['num_mths']=np.where(df.curmth.eq(0),mnth_diff-1,mnth_diff-2)
df

,mydate1,mydate2,curmth,num_mths
0,2019-07-17,2019-12-31,0,4
1,2019-03-18,2019-12-31,0,8
2,2019-03-27,2019-12-31,1,7
